# Sentiment Analysis
Sentiment analysis of song lyrics using pretrained models from HuggingFace.co

Models:
- [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
- [oliverguhr/german-sentiment-bert](https://huggingface.co/oliverguhr/german-sentiment-bert)
- [cardiffnlp/twitter-roberta-base-sentiment](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment)
- [deepset/gbert-base-germandpr-reranking](https://huggingface.co/deepset/gbert-base-germandpr-reranking)


# Setup

In [ ]:
!pip install torch torchvision torchaudio

# Get max sequence length
The models have the following max sequence lengths:
- distilbert-base-uncased-finetuned-sst-2-english: 512
- oliverguhr/german-sentiment-bert: 512
- cardiffnlp/twitter-roberta-base-sentiment: 514
- deepset/gbert-base-germandpr-reranking: 512

Lyrics need to be preprocessed to fit the max sequence length.

In [9]:
from transformers import AutoModel

models = [
    "distilbert-base-uncased-finetuned-sst-2-english",
    "oliverguhr/german-sentiment-bert",
    "cardiffnlp/twitter-roberta-base-sentiment",
    "deepset/gbert-base-germandpr-reranking"
]

for model_name in models:
    model = AutoModel.from_pretrained(model_name)
    print(f"{model_name}: {model.config.max_position_embeddings}")

Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing DistilBertModel: ['pre_classifier.weight', 'classifier.bias', 'classifier.weight', 'pre_classifier.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


distilbert-base-uncased-finetuned-sst-2-english: 512


Some weights of the model checkpoint at oliverguhr/german-sentiment-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


oliverguhr/german-sentiment-bert: 512


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment were not used when initializing RobertaModel: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

cardiffnlp/twitter-roberta-base-sentiment: 514


Some weights of the model checkpoint at deepset/gbert-base-germandpr-reranking were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


deepset/gbert-base-germandpr-reranking: 512


# Load lyrics

In [23]:
import os
import json

processing1_folder_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'data', 'processed', 'processing2'))
file_names = os.listdir(processing1_folder_path)
file_paths = [os.path.join(processing1_folder_path, file) for file in file_names]

data = []
for file_path in file_paths:
    with open(file_path, 'r') as f:
        artist = json.load(f)
        data.append({
            'artist': artist[0]['artist'],
            'songs': artist
        })

print(f"Loaded {len(data)} artists")

Loaded 14 artists


# Functions

In [41]:
def text_to_chunks(text):
    """
    Splits text into chunks of max length 512
    :param text: Text to split
    :return: List of chunks
    """
    chunks = []
    chunk = ""
    for word in text.split():
        if len(chunk) + len(word) + 1 <= 512:
            chunk += f" {word}"
        else:
            chunks.append(chunk)
            chunk = word
    chunks.append(chunk)
    return chunks

def store_to_output(filename, data, subfolder = None):
    """
    Stores data to output folder as JSON file
    :param filename: Name of the file
    :param data: Data to store
    :param subfolder: Subfolder to store file in
    :return:
    """
    # Output folder
    if subfolder:
        # Create subfolder if it does not exist
        if not os.path.exists(os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'data', 'output', subfolder))):
            os.makedirs(os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'data', 'output', subfolder)))
        full_filepath = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'data', 'output', subfolder, filename + '.json'))
    else:
        full_filepath = os.path.abspath(os.path.join(os.getcwd(), '..', '..', 'data', 'output', filename + '.json'))

    # Object to JSON
    json_data = json.dumps(data, ensure_ascii=False, indent=4)

    # Write to file encoded as UTF-8
    with open(full_filepath, "w", encoding="utf-8") as file:
        file.write(json_data)


    return

# Run Sentiment Analysis

In [42]:
from transformers import pipeline

# Init models
en_distilbert = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", return_all_scores=False)
en_roberta = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment", return_all_scores=False)
de_gbert = pipeline("sentiment-analysis", model="deepset/gbert-base-germandpr-reranking", return_all_scores=False)
de_bert = pipeline("sentiment-analysis", model="oliverguhr/german-sentiment-bert", return_all_scores=False)

print(en_distilbert("I love you"))
print(en_roberta("I love you"))
print(de_gbert("Ich liebe dich"))
print(de_bert("Ich liebe dich"))

# function to run sentiment analysis on a text but split it into chunks first and return the average sentiment and label name from model response
def sentiment_analysis(text, model):
    chunks = text_to_chunks(text)
    sentiments = []
    for chunk in chunks:
        sentiment = model(chunk)[0]
        sentiments.append(sentiment)
    avg_score = sum([sentiment['score'] for sentiment in sentiments]) / len(sentiments)
    label = sentiments[0]['label']
    return {
        'score': avg_score,
        'label': label
    }

overall_progress = 0
for artist in data:
    overall_progress += 1
    for song in artist['songs']:
        if song['language'] == 'en':
            song['sentiment'] = {
                'en_distilbert': sentiment_analysis(song['lyrics'], en_distilbert),
                'en_roberta': sentiment_analysis(song['lyrics'], en_roberta),
                'de_gbert': None,
                'de_bert': None
            }
        elif song['language'] == 'de':
            song['sentiment'] = {
                'en_distilbert': None,
                'en_roberta': None,
                'de_gbert': sentiment_analysis(song['lyrics'], de_gbert),
                'de_bert': sentiment_analysis(song['lyrics'], de_bert),
            }
        else:
            song['sentiment'] = {
                'en_distilbert': None,
                'en_roberta': None,
                'de_gbert': None,
                'de_bert': None
            }

        print(f"Processed {song['title']} by {artist['artist']}")

    print(f"Progress: {overall_progress}/{len(data)}")
    store_to_output(artist['artist'], artist, 'sentiment_analysis')


/Users/joba/Documents/GitHub/lyrics-analysis/venv/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[{'label': 'POSITIVE', 'score': 0.9998656511306763}]
[{'label': 'LABEL_2', 'score': 0.9557049870491028}]
[{'label': '0', 'score': 0.9891400337219238}]
[{'label': 'positive', 'score': 0.9846151471138}]
Processed God’s Plan by Drake
Processed In My Feelings by Drake
Processed Hotline Bling by Drake
Processed One Dance by Drake
Processed Hold On, We’re Going Home by Drake
Processed Know Yourself by Drake
Processed Back to Back by Drake
Processed All Me by Drake
Processed From Time by Drake
Processed Fake Love by Drake
Processed 0 to 100 / The Catch Up by Drake
Processed The Motto by Drake
Processed Pound Cake / Paris Morton Music 2 by Drake
Processed Started from the Bottom by Drake
Processed Marvins Room by Drake
Processed HYFR (Hell Ya Fucking Right) by Drake
Processed Passionfruit by Drake
Processed Nice For What by Drake
Processed Energy by Drake
Processed Child’s Play by Drake
Processed Too Good by Drake
Processed Furthest Thing by Drake
Processed Jungle by Drake
Processed Over My De